# A14 - DCT studies.
Stough, DIP
In conjunction with the [paper activity](https://docs.google.com/document/d/1fZOkp1fE5Q0Z64qmcuveyZX86h0T9McICnlsdeqWha8/edit?usp=sharing).

In [1]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np

from waveletUtil import (makeStandardMatrix,
                         makeHaarMatrix,
                         makeDCTMatrix,
                         makeRandomBasis,
                         makeKLTBasis)

In [2]:
def arrInfo(arr):
    return arr.shape, arr.min(), arr.max(), arr.dtype

In [3]:
I = plt.imread('shamokin.png')[...,:3] #remove alpha chan
arrInfo(I)


((512, 512, 3), 0.003921569, 1.0, dtype('float32'))

In [4]:
plt.figure(figsize=(4,4))
plt.imshow(I);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
DCT = makeDCTMatrix(512)
I = plt.imread('shamokin.png')[...,:3] #remove alpha chan
print(arrInfo(DCT))
print(arrInfo(I))
T = np.zeros(I.shape[:2] + (3,))
for chan in range(3):
    T[...,chan] = np.matmul(DCT, np.matmul(I[..., chan], DCT.transpose()))
print(arrInfo(T))
plt.figure()
plt.imshow(DCT)


((512, 512), -0.06249970586309854, 0.06249970586309854, dtype('float64'))
((512, 512, 3), 0.003921569, 1.0, dtype('float32'))
((512, 512, 3), -25.365031308369595, 267.5650407434223, dtype('float64'))


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
plt.figure()
plt.hist(T[...,0].ravel(),np.arange(-2,2,.01))
print(arrInfo(T))
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

((512, 512, 3), -25.365031308369595, 267.5650407434223, dtype('float64'))


In [22]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets


def lossly(keep=50):
    Tc = np.zeros(T.shape)
    DCT = makeDCTMatrix(512)
    Tr = np.zeros(T.shape)

    for chan in range(3):
        cutoff = np.percentile(np.abs(T[...,chan].ravel()), keep)
        mask = (np.abs(T[...,0]) >= cutoff)
        Tc[mask,chan] = T[mask,chan]
    
    
  
    for chan in range(3):
        Tr[...,chan] = np.matmul(DCT.T, np.matmul(Tc[..., chan], DCT))

    print(arrInfo(Tr))
    plt.clf()
    plt.imshow(np.clip(Tr,0,1))
    print((~mask).sum()) # Falses
    print((mask).sum()) # Trues

In [23]:
#DCT.shape
##print(arrInfo(Tc))
#print(cutoff)
#print((~mask).sum()) # Falses
#print((mask).sum()) # Trues

In [24]:
plt.figure()
interact(lossly, keep=(0,100))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=50, description='keep'), Output()), _dom_classes=('widget-interact',))

<function __main__.lossly(keep=50)>